In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline
pd.set_option('display.max_columns', 40)

## Import Data

As usual, we will first import our data. We also take an excerpt figure from the M5 Competition handbook to show the breakdown of how the data is categorized:

<figure>
    <img src = "Supplementals/M5_Competition_Categories.png", style="width:90%">
    <figcaption><b>Fig.1: M5 Data Categorical Breakdown</b></figcaption>
</figure>

### Objective:
<i>The objective of the M5 forecasting competition is to advance the theory and practice of forecasting by identifying the method(s) that provide the most accurate point forecasts for each of the 42,840 time series of the competition. I addition, to elicit information to estimate the uncertainty distribution of the realized values of these series as precisely as possible. 
To that end, the participants of M5 are asked to <b>provide 28 days ahead point forecasts (PFs)</b> for all the series of the competition, as well as the corresponding median and 50%, 67%, 95%, and 99% prediction intervals (PIs).</i>


In [2]:
#sales_train_val = pd.read_csv('sales_train_validation.csv')
calendar = pd.read_csv('calendar.csv')
sales_train_eval = pd.read_csv('sales_train_evaluation.csv')
sample_submission = pd.read_csv('sample_submission.csv')
sell_prices = pd.read_csv('sell_prices.csv')

In [3]:
# We can take a look at the desired format by examining the first 10 entries from the sample submission
sample_submission.head(10)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,HOBBIES_1_006_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,HOBBIES_1_007_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,HOBBIES_1_008_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,HOBBIES_1_009_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,HOBBIES_1_010_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
#Ensure that calendar is ordered properly

calendar = calendar.sort_values(by = ['wm_yr_wk', 'wday'])
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,NaN,1,1,1
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,NaN,1,0,0
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,NaN,1,1,1
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,NaN,1,1,1
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,NaN,1,1,0


In [5]:
# I'll first modify the sell_prices dataframe so that it explicitly lists the relevant Year and Week of the year as encoded in
# the wm_yr_wk column. To do this, I'll define two functions that can extract the Year and Week for each wm_yr_wk value. We note
# that this is strictly for visualization purposes.

def year_extract(wk_id):
    year = (wk_id - (wk_id % 100))/100 - 100
    return int(year + 2000)

def week_extract(wk_id):
    week = wk_id % 100
    return week

sell_prices['Year'] = sell_prices['wm_yr_wk'].apply(year_extract)
sell_prices['Week'] = sell_prices['wm_yr_wk'].apply(week_extract)

# Check out the new sell_prices dataframe
sell_prices.head(10)

,store_id,item_id,wm_yr_wk,sell_price,Year,Week
0,CA_1,HOBBIES_1_001,11325,9.58,2013,25
1,CA_1,HOBBIES_1_001,11326,9.58,2013,26
2,CA_1,HOBBIES_1_001,11327,8.26,2013,27
3,CA_1,HOBBIES_1_001,11328,8.26,2013,28
4,CA_1,HOBBIES_1_001,11329,8.26,2013,29
5,CA_1,HOBBIES_1_001,11330,8.26,2013,30
6,CA_1,HOBBIES_1_001,11331,8.26,2013,31
7,CA_1,HOBBIES_1_001,11332,8.26,2013,32
8,CA_1,HOBBIES_1_001,11333,8.26,2013,33
9,CA_1,HOBBIES_1_001,11334,8.26,2013,34


# Models

The model architecture will influence how we want to structure the training data. There are of course numerous ways to go about doing this but we will propose a few here. A barebones template for our models will be that it takes some collection of values as input and outputs a prediction for the next 28 day sale values for that particular item. 

One of the first types of model structures that comes to mind is a function $model: \mathbb{R}^{n+1} \to \mathbb{R}^{28}$ that takes as input the $n+1$ previous days of sale values and outputs a prediction for the next 28 days:

\begin{equation}
    pred(t+1), pred(t+2), ..., pred(t+28) = model(obs(t), obs(t-1),...,obs(t-n)),
\end{equation}

where $pred(t+j)$ denotes the prediction for the $j$-th next day from which the model wants to perform a prediction and $obs(t-j)$ denotes the observation made on the $j$-th previous day. We note that such a model assumes a causal relationship between past sales and future sales, which for this model and most that we will consider we argue is a fair assumption to make. In such a model shown above, we explicitly assign a weighting to the prior $n+1$ days and zero to all others. We can of course modify which days in the past we would like to include as input into our model, which can be played around with to optimize the model.

### Encoding Categorical Variables

So far, this is an okay start, but it's useful to also encode categorical variables such as any events taking place over the next 4 weeks or what store / category the item belonged to. Suppose that the collection of all possible events had a size (or cardinality) of $n$. Then, for each such unique event, we can assign a number $g \in \mathbb{Z}_{n+1}$ that ranges from $1$ to $n$ to uniquely identify it. In this case, we consider $0$ to indicate 'no event'. Then, we can encode this into our model by specifying the event value for a given period of time, such as a week. 

If we assign a single value to a week, then we can define $model: \mathbb{Z}_{n+1}^4 \times \mathbb{R}^{n+1} \to \mathbb{R}^{28}$ where we have introduced the notation

\begin{equation}
    \mathbb{Z}^{j}_{m} := \underbrace{\mathbb{Z}_{m} \times ... \times \mathbb{Z}_{m}}_{\text{j times}}.
\end{equation}
### Loss Function 

To train a model, one standard way to do this is by formalizing a measure of model accuracy. Such a measure is typically encoded in quantities known as loss functions that we aim to minimize. These functions assign a penalty to predicted values that deviate from the observed values.

One common loss function is the <b> squared-error loss</b> function or <b> L2-Loss </b> function. It is given by

\begin{equation}
    L2(y,\hat{y}) = \sum_{i=1}^{28}(y_i - \hat{y}_i)^2,
\end{equation}

where $y_i$ denotes the observation for the $i$-th day and $\hat{y}_i$ denotes the prediction for the $i$-th day.









## Model 1: Neural Network

There are various neural network architectures that we can consider. One of the most standard and familiar ones are feed-forward neural networks where nodes are only connected to adjacent layers. An architecture I have proposed is provided below:

<figure>
    <img src = "Supplementals/NeuralNetworkStructure.png" width = "1000" height = "600">
    <figcaption><b>Fig.2: Standard Feed-Forward Neural Network Architecture with Listed Inputs</b></figcaption>
</figure>

### Parameters: 

Suppose that we only consider data from $n$ days in the past. We denote prediction day $j$ as the date $j$ days in the future from the day of measurement. We define the following parameters that enter into our input layer:

<ul>
    <li> <b>S</b>: Store ID (0-9) </li>
    <li> <b>item_id</b>: Item ID </li>
    <li> <b>F_Event$_j$</b>: The Future Event value that will occur on prediction day $j$</li>
    <li> <b>F_Snap$_j$</b>: Snap Value that will occur on future day $j$ (0 or 1)</li>
    <li> <b>F_Yr_wk$_j$</b>: Year-Week Number that occurred on future day $j$ </li>
    <li> <b>F_Wk_day$_j$</b>: Week-Day Value that occurred on future day $j$ (1-7) </li>
    <li> <b>F_Price$_j$</b>: The future sell price that occurred on future day $j$ </li>
    <li> <b>tar$_j$</b>: Number of Sales that occurred on (future) target day $j$ </li>
    <li> <b>Event$_i$</b>: The Event value that occurred on prior day $i$</li>
    <li> <b>Snap$_i$</b>: Snap Value that occurred on prior day $i$ (0 or 1)</li>
    <li> <b>Yr_wk$_i$</b>: Year-Week Number that occurred on prior day $i$ </li>
    <li> <b>Wk_day$_i$</b>: Week-Day Value that occurred on prior day $i$ (1-7) </li>
    <li> <b>Price$_i$</b>: Sell Price that occurred on prior day $i$ </li>
    <li> <b>d$_i$</b>: Number of Sales that occurred on prior day $i$ </li>
</ul>

In [6]:
# We'll first transform the event labels to categorical labels by assigning a unique integer to every unique event.
events = calendar['event_name_1'].unique()
event_map = {events[0]: 0}

# Exclude nan
events = events[1::] 

i=1
for word in events:
    
    event_map[word] = i
    i = i + 1
    
def event_value(event_name1,event_name2):
    # Returns a 4 digit number where the first 2 digits indicate event value of event_name_1 and last 2 indicate
    # event value of event_name_2.
    
    value = event_map[event_name1]*(10**2) + event_map[event_name2]
    
    return value

calendar['EventValue'] = calendar.apply(lambda x: event_value(x.event_name_1,x.event_name_2), axis = 1)
calendar = calendar.rename(mapper= {'d': 'day_id'},axis = 1)
calendar

,date,wm_yr_wk,weekday,wday,month,year,day_id,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,EventValue
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1,0
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,NaN,1,1,1,0
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,NaN,1,0,0,0
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,NaN,1,1,1,0
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,NaN,1,1,1,100
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,NaN,1,1,0,0


In [7]:
# Assign Numeric Values to Categorical Variables

department_map = {'HOBBIES_1': 11, 'HOBBIES_2':12, 'HOUSEHOLD_1' : 21, 'HOUSEHOLD_2':22, 'FOODS_1' : 31,
       'FOODS_2':32, 'FOODS_3':33}

store_map = {'CA_1': 1, 'CA_2': 2, 'CA_3':3, 'CA_4':4, 'TX_1':5, 'TX_2':6, 'TX_3':7, 'WI_1':8,
       'WI_2':9, 'WI_3':10}

def dept_value(dept):
    
    return department_map[dept]

def store_value(store):
    
    return store_map[store]

sales_train_eval = sales_train_eval.rename(mapper = {'store_id': 'Store', 'dept_id': 'Dept', 'item_id' : 'Item', 'cat_id':'Category'},  axis = 1)

sales_train_eval['DEPT_ID'] = sales_train_eval['Dept'].apply(dept_value)
sales_train_eval['STORE_ID'] = sales_train_eval['Store'].apply(store_value)

# We can also convert a column into a 'category' and then assign a numeric value by extracting the category codes
sales_train_eval['ITEM_ID'] = sales_train_eval['Item'].astype('category').cat.codes
sales_train_eval[sales_train_eval['ITEM_ID'] == 1437]

,id,Item,Dept,Category,Store,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,...,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,DEPT_ID,STORE_ID,ITEM_ID
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,2,1,2,2,1,0,2,4,0,0,0,0,3,3,0,1,11,1,1437
3049,HOBBIES_1_001_CA_2_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_2,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,2,2,0,2,0,2,2,0,2,0,1,11,2,1437
6098,HOBBIES_1_001_CA_3_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_3,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,4,1,1,0,0,0,2,6,0,1,0,2,1,0,1,0,11,3,1437
9147,HOBBIES_1_001_CA_4_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,2,1,0,0,1,0,3,1,1,1,0,1,2,2,11,4,1437
12196,HOBBIES_1_001_TX_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_1,TX,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,2,0,1,0,0,0,0,2,1,0,2,1,0,1,11,5,1437
15245,HOBBIES_1_001_TX_2_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_2,TX,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0,0,0,2,0,0,0,1,11,6,1437
18294,HOBBIES_1_001_TX_3_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_3,TX,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,0,0,1,0,1,0,3,0,0,3,1,1,2,1,11,7,1437
21343,HOBBIES_1_001_WI_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,WI_1,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,1,0,2,0,0,0,0,1,2,11,8,1437
24392,HOBBIES_1_001_WI_2_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,WI_2,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,11,9,1437
27441,HOBBIES_1_001_WI_3_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,WI_3,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,10,1437


In [8]:
# Ensure that every item was in stock at every store

unique_items = sales_train_eval['ITEM_ID'].nunique()
print("Number of Distinct Items: ", unique_items)
sales_train_eval['Store'].value_counts()

Number of Distinct Items:  3049


WI_2    3049
WI_1    3049
TX_3    3049
TX_2    3049
WI_3    3049
TX_1    3049
CA_1    3049
CA_2    3049
CA_4    3049
CA_3    3049
Name: Store, dtype: int64

In [9]:
# Adding ITEM_ID and STORE_ID columns to the sell_prices dataframe

sell_prices = sell_prices.rename(mapper = {'store_id': 'Store', 'dept_id': 'Dept', 'item_id' : 'Item', 'cat_id':'Category'},  axis = 1)

sell_prices['STORE_ID'] = sell_prices['Store'].apply(store_value)

# We can also convert a column into a 'category' and then assign a numeric value by extracting the category codes
sell_prices['ITEM_ID'] = sell_prices['Item'].astype('category').cat.codes
sell_prices

,Store,Item,wm_yr_wk,sell_price,Year,Week,STORE_ID,ITEM_ID
0,CA_1,HOBBIES_1_001,11325,9.58,2013,25,1,1437
1,CA_1,HOBBIES_1_001,11326,9.58,2013,26,1,1437
2,CA_1,HOBBIES_1_001,11327,8.26,2013,27,1,1437
3,CA_1,HOBBIES_1_001,11328,8.26,2013,28,1,1437
4,CA_1,HOBBIES_1_001,11329,8.26,2013,29,1,1437
5,CA_1,HOBBIES_1_001,11330,8.26,2013,30,1,1437
6,CA_1,HOBBIES_1_001,11331,8.26,2013,31,1,1437
7,CA_1,HOBBIES_1_001,11332,8.26,2013,32,1,1437
8,CA_1,HOBBIES_1_001,11333,8.26,2013,33,1,1437
9,CA_1,HOBBIES_1_001,11334,8.26,2013,34,1,1437


### Opening Days

I noticed that not all items have information on their sell prices dating all the way back to day value d_1. It appears that new items were added to the list as they came into stock. Hence, the goal is to establish the "opening day" of these items and therefore use that day value as where to begin for generating the training entries. We also note that not all stores obtained the item in stock on the same week. If we don't do this, then there is plenty of data in sales_train_eval where prior to the opening day the number of sales on day <i>d_i</i> is listed as 0.

In [10]:
# Specify the desired ITEM_ID in item_id, STORE_ID in store_id and you will obtain the opening week and day values as well as the local sales around
# opening day.

item_id = 1427
store_id = 1

def opening_day(item_id, store_id):
    
    first_week = sell_prices[(sell_prices['ITEM_ID'] == item_id) & (sell_prices['STORE_ID'] == store_id)].sort_values(by = 'wm_yr_wk')['wm_yr_wk'].iloc[0]
    
    first_day = calendar[calendar['wm_yr_wk'] == first_week]['day_id'].iloc[0]
    
    return first_week, first_day

first_week, first_day = opening_day(item_id, store_id)

print("Opening Day:", first_day)
print("Opening Week:", first_week)

def day_from_id(day_id):
    
    day_mags = list(day_id)[2::]
    max_power = len(day_mags)
    
    day = 0
    
    for power in range(max_power):
        
        day = day + int(day_mags[power])*(10**(max_power - power - 1))
        
    return day

local_days = ['id', 'Item', 'Dept', 'Category', 'Store', 'DEPT_ID', 'STORE_ID', 'ITEM_ID']

for local in range(20):
    
    open_day = day_from_id(first_day)
    
    if open_day - 5 <= 0:
        day = open_day
    else:
        day =  open_day - 5 + local
        
    local_days.append('d_' + str(day))

# Check The Local Sale Values for item_unit near Opening Day
sales_train_eval[sales_train_eval['ITEM_ID'] == item_id][local_days]

Opening Day: d_477
Opening Week: 11217


,id,Item,Dept,Category,Store,DEPT_ID,STORE_ID,ITEM_ID,d_472,d_473,d_474,d_475,d_476,d_477,d_478,d_479,d_480,d_481,d_482,d_483,d_484,d_485,d_486,d_487,d_488,d_489,d_490,d_491
3039,FOODS_3_818_CA_1_evaluation,FOODS_3_818,FOODS_3,FOODS,CA_1,33,1,1427,0,0,0,0,0,3,2,4,0,2,2,4,2,6,2,3,3,2,5,3
6088,FOODS_3_818_CA_2_evaluation,FOODS_3_818,FOODS_3,FOODS,CA_2,33,2,1427,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9137,FOODS_3_818_CA_3_evaluation,FOODS_3_818,FOODS_3,FOODS,CA_3,33,3,1427,0,0,0,0,1,1,1,2,2,2,1,1,2,2,4,0,2,5,6,0
12186,FOODS_3_818_CA_4_evaluation,FOODS_3_818,FOODS_3,FOODS,CA_4,33,4,1427,0,0,0,0,1,1,2,3,3,1,4,1,0,1,3,0,1,5,2,1
15235,FOODS_3_818_TX_1_evaluation,FOODS_3_818,FOODS_3,FOODS,TX_1,33,5,1427,1,0,0,1,0,2,1,0,0,0,0,1,0,0,2,2,0,0,4,0
18284,FOODS_3_818_TX_2_evaluation,FOODS_3_818,FOODS_3,FOODS,TX_2,33,6,1427,2,3,1,1,0,1,2,1,1,1,0,0,4,1,4,2,1,2,6,4
21333,FOODS_3_818_TX_3_evaluation,FOODS_3_818,FOODS_3,FOODS,TX_3,33,7,1427,0,0,3,0,1,3,1,1,0,2,1,0,2,1,1,0,1,1,1,2
24382,FOODS_3_818_WI_1_evaluation,FOODS_3_818,FOODS_3,FOODS,WI_1,33,8,1427,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27431,FOODS_3_818_WI_2_evaluation,FOODS_3_818,FOODS_3,FOODS,WI_2,33,9,1427,0,0,1,1,0,0,0,1,2,0,1,0,1,0,0,0,0,0,0,0
30480,FOODS_3_818_WI_3_evaluation,FOODS_3_818,FOODS_3,FOODS,WI_3,33,10,1427,0,1,2,4,2,5,2,2,2,2,1,2,2,0,1,1,3,3,0,1


## Constructing our Training Entries and Target Data

In [11]:
sell_prices.sort_values(['wm_yr_wk', 'ITEM_ID']).head(50)

sell_prices[(sell_prices['ITEM_ID'] == 0) & (sell_prices['STORE_ID'] == 7)]

,Store,Item,wm_yr_wk,sell_price,Year,Week,STORE_ID,ITEM_ID
4476635,TX_3,FOODS_1_001,11102,2.00,2011,2,7,0
4476636,TX_3,FOODS_1_001,11103,2.00,2011,3,7,0
4476637,TX_3,FOODS_1_001,11104,2.00,2011,4,7,0
4476638,TX_3,FOODS_1_001,11105,2.00,2011,5,7,0
4476639,TX_3,FOODS_1_001,11106,2.00,2011,6,7,0
4476640,TX_3,FOODS_1_001,11107,2.00,2011,7,7,0
4476641,TX_3,FOODS_1_001,11108,2.00,2011,8,7,0
4476642,TX_3,FOODS_1_001,11109,2.00,2011,9,7,0
4476643,TX_3,FOODS_1_001,11110,2.00,2011,10,7,0
4476644,TX_3,FOODS_1_001,11111,2.00,2011,11,7,0


In [33]:
# Constructing a Training Entry - Currently Drastically Inefficient, Will need to vectorize via Pandas methods

def snap_store(store_id):
    
    if store_id in [1,2,3,4]:
        return 'snap_CA'
    
    elif store_id in [5,6,7]:
        return 'snap_TX'
    
    else:
        return 'snap_WI'
    
def df_item_struct(item_ID, store_id, today, train_days = 31, future_days = 28):
    
    dataframe = sales_train_eval[(sales_train_eval['ITEM_ID'] == item_ID) & (sales_train_eval['STORE_ID'] == store_id)].iloc[:,6::]
    dataframe_days = dataframe.iloc[:,today - train_days:today + future_days]
    dataframe = pd.concat([dataframe_days, dataframe.iloc[:,1941:1944]], join = 'outer', axis = 1)
    
    init_day = 'd_' + str(today - train_days + 1)
    dataframe['INIT_DAY'] = init_day
    dataframe['TODAY'] = 'd_' + str(today)
    
    for day in range(train_days):
        
        day_id = 'd_' + str(today - train_days + 1 + day)
        
        event_col = 'Event_' + str(day)
        dataframe[event_col] = calendar[calendar['day_id'] == day_id]['EventValue'].iloc[0]
        
        year_week_col = 'Yr_wk_' + str(day)
        dataframe[year_week_col] = calendar[calendar['day_id'] == day_id]['wm_yr_wk'].iloc[0]
        
        week_day_col = 'Wk_day_' + str(day)
        dataframe[week_day_col] = calendar[calendar['day_id'] == day_id]['wday'].iloc[0]
        
        price_col = 'Price_' + str(day)
        dataframe[price_col] = sell_prices[(sell_prices['wm_yr_wk'] == dataframe[year_week_col].iloc[0]) & 
                                           (sell_prices['ITEM_ID'] == item_ID) & 
                                           (sell_prices['STORE_ID'] == store_id)]['sell_price'].iloc[0]
        
        snap_col = 'Snap_' + str(day)
        dataframe[snap_col] = calendar[calendar['day_id'] == day_id][snap_store(store_id)].iloc[0]
    
    for day in range(future_days):
        
        day_id = 'd_' + str(today + 1 + day)
        
        event_col = 'F_Event_' + str(day)
        dataframe[event_col] = calendar[calendar['day_id'] == day_id]['EventValue'].iloc[0]
        
        year_week_col = 'F_Yr_wk_' + str(day)
        dataframe[year_week_col] = calendar[calendar['day_id'] == day_id]['wm_yr_wk'].iloc[0]
        
        week_day_col = 'F_Wk_day_' + str(day)
        dataframe[week_day_col] = calendar[calendar['day_id'] == day_id]['wday'].iloc[0]
        
        price_col = 'F_Price_' + str(day)
        dataframe[price_col] = sell_prices[(sell_prices['wm_yr_wk'] == dataframe[year_week_col].iloc[0]) & 
                                           (sell_prices['ITEM_ID'] == item_ID) & 
                                           (sell_prices['STORE_ID'] == store_id)]['sell_price'].iloc[0]
        
        snap_col = 'F_Snap_' + str(day)
        dataframe[snap_col] = calendar[calendar['day_id'] == day_id][snap_store(store_id)].iloc[0]
        
    entry = dataframe.reindex(sorted(dataframe.columns), axis=1)
    
    # Redefine the Prior Day Sales and Target Sales Column so that it reflects the desired format
    sales_map = {}
    
    for day in range(future_days):
        
        target_id = 'Tar_' + str(day)
        day_id = 'd_' + str(today + day + 1)
        sales_map[day_id] = target_id
        
    for day in range(train_days):
        
        train_id = 'D_' + str(day)
        day_id = 'd_' + str(today - train_days + 1 + day)
        sales_map[day_id] = train_id
    
    entry = entry.rename(mapper = sales_map, axis = 1)
    
    return entry

test = df_item_struct(item_ID = 0, store_id = 1, today = 930, train_days = 31, future_days = 28)

test

,DEPT_ID,Event_0,Event_1,Event_10,Event_11,Event_12,Event_13,Event_14,Event_15,Event_16,Event_17,Event_18,Event_19,Event_2,Event_20,Event_21,Event_22,Event_23,Event_24,Event_25,...,Tar_8,Tar_9,Tar_10,Tar_11,Tar_12,Tar_13,Tar_14,Tar_15,Tar_16,Tar_17,Tar_18,Tar_19,Tar_20,Tar_21,Tar_22,Tar_23,Tar_24,Tar_25,Tar_26,Tar_27
1612,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1800,0,0,...,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,2,0,0


In [44]:
%%time

def train_target_data_constructor(num_items, final_day, train_days = 31, future_days = 28, stepsize = 1):
    
    # Initial Dataframe
    _, open_day_id = opening_day(0, store_id = 1)
    open_day = day_from_id(open_day_id)
    today = open_day + train_days
    print("Start Date: ", today)
    print("Total Entries: ", final_day - (today+future_days))
    train_target_data = df_item_struct(item_ID = 0, store_id = 1, today = today, train_days = train_days, future_days = future_days)
    
    today = today + stepsize
    while today + future_days <= final_day:
        
        new_entry = df_item_struct(item_ID = 0, store_id = 1, today = today, train_days = train_days, future_days = future_days)
        train_target_data = pd.concat([train_target_data, new_entry])
        today = today + stepsize
    
    """
    for item in range(num_items):
        
        while today + future_days <= final_day:
        
    """
    
    return train_target_data


tt_data = train_target_data_constructor(1, 75)    
tt_data

Start Date:  32
Total Entries:  15
Wall time: 59.3 s


In [46]:
tt_data['INIT_DAY']

1612     d_2
1612     d_3
1612     d_4
1612     d_5
1612     d_6
1612     d_7
1612     d_8
1612     d_9
1612    d_10
1612    d_11
1612    d_12
1612    d_13
1612    d_14
1612    d_15
1612    d_16
1612    d_17
Name: INIT_DAY, dtype: object